# **Importing data from kaggel**

In [2]:
! pip install -q kaggle --quiet

In [3]:
# ! pip install gensim --quiet

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"goutammunda","key":"45045cb644adc5976bb896eaf8890b63"}'}

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
! kaggle datasets download -d saurabhshahane/fake-news-classification

Dataset URL: https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification
License(s): Attribution 4.0 International (CC BY 4.0)
fake-news-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


# **UnZip the dataset**

In [7]:
import zipfile
zip_ref = zipfile.ZipFile('/content/fake-news-classification.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# **Importing Libraries**

In [38]:
import re
import nltk
import tensorflow
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [9]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Simple data loading and cleaning**

In [11]:
df = pd.read_csv('/content/WELFake_Dataset.csv')

In [12]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [13]:
print(df.shape)

(72134, 4)


In [14]:
df.isnull().sum()

,0
Unnamed: 0,0
title,558
text,39
label,0


In [15]:
df = df.dropna()

In [16]:
df.shape

(71537, 4)

In [17]:
df = df.drop(['Unnamed: 0'], axis=1)

In [18]:
df = df.reset_index(drop=True)

In [19]:
df.tail()

,title,text,label
71532,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
71533,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
71534,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
71535,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0
71536,Goldman Sachs Endorses Hillary Clinton For Pre...,Goldman Sachs Endorses Hillary Clinton For Pre...,1


# **Text Pre-Processing**

In [20]:
stemmer = SnowballStemmer(language='english')
corpus = []

def preprocess_txt(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # removes URLs
    text = re.sub(r'<.*?>', '', text)  # removes HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # removes non-alphabetic characters
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))

    # Removing stop words
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]

    # Removing duplicates while preserving order
    unique_tokens = []
    for token in tokens:
        if token not in unique_tokens:
            unique_tokens.append(token)

    preprocessed_text = ' '.join(unique_tokens)
    corpus.append(preprocessed_text)
    return preprocessed_text

In [21]:
print(len(corpus))

0


In [22]:
# df['preprocessed_title'] = df['title'].apply(preprocess_txt)
df['preprocessed_text'] = df['text'].apply(preprocess_txt)

In [23]:
print(f"len of corpus: {len(corpus)}")

len of corpus: 71537


In [24]:
df.head()

,title,text,label,preprocessed_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,comment expect barack obama member fyf fukyofl...
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,demonstr gather last night exercis constitut p...
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,dozen polit activ pastor came privat dinner fr...
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,rs sarmat missil dub satan replac ss fli mile ...
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,say one time someon su southern poverti law ce...


In [25]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# df.to_csv('/content/drive/MyDrive/Processed_WELFake_Dataset.csv')

# **WOrd Embeddings**

In [27]:
vocab_size = 10000

onehot=[one_hot(words,vocab_size)for words in corpus]

- `vocab_size = 10000`
This sets the size of the vocabulary to 10,000. This means that Keras will map each word to an integer index within the range [0, 9999]
- `one_hot(words, vocab_size)`
The one_hot function is a Keras utility that converts a word (or sequence of words) into a list of integer indices. Each word in the text is hashed to an integer within the range defined by vocab_size

In [28]:
print(f"Sentence at index 1 : {corpus[1]}")
print(f"One Hot Representation: {onehot[1]}")  # indices of the one hot encoded words

Sentence at index 1 : demonstr gather last night exercis constitut protect right peac protest order rais issu creat chang loretta lynch aka eric holder skirt
One Hot Representation: [9727, 516, 7965, 2593, 3455, 2189, 1034, 9077, 5032, 9654, 5535, 5820, 8283, 8889, 3357, 9152, 5447, 5991, 8919, 8830, 1308]


- all the sentences are of variable length we have to pad them to make them of equal lengths.

## Padding Sequence

In [84]:
sent_length = 500
padded_onehot = pad_sequences(onehot, padding='post',maxlen=sent_length)
print(padded_onehot)

[[1654 9228 8341 ...    0    0    0]
 [9727  516 7965 ...    0    0    0]
 [2028 9159 6860 ...    0    0    0]
 ...
 [1180 6661 3337 ...    0    0    0]
 [9894  406 9119 ...    0    0    0]
 [2558 9002 7230 ...    0    0    0]]


In [85]:
padded_data = np.array(padded_onehot)
label = np.array(df['label'])

print(padded_data.shape)
print(label.shape)

(71537, 500)
(71537,)


# Spliting the dataset for Train and Testing

In [86]:
X_train, X_test, y_train, y_test = train_test_split(padded_onehot, label, test_size=0.2, random_state=42)
print(f"training data shape {X_train.shape}")
print(f"testing data shape {X_test.shape}")

training data shape (57229, 500)
testing data shape (14308, 500)


In [87]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [88]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 26s 26ms/step - accuracy: 0.5078 - loss: 0.6960 - val_accuracy: 0.5492 - val_loss: 0.6843
Epoch 2/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - accuracy: 0.5190 - loss: 0.6873 - val_accuracy: 0.5226 - val_loss: 0.6777
Epoch 3/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.5233 - loss: 0.6689 - val_accuracy: 0.5142 - val_loss: 0.6733
Epoch 4/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.5471 - loss: 0.6555 - val_accuracy: 0.5554 - val_loss: 0.6804
Epoch 5/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 42s 26ms/step - accuracy: 0.5893 - loss: 0.6477 - val_accuracy: 0.5159 - val_loss: 0.6855
Epoch 6/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.5304 - loss: 0.6612 - val_accuracy: 0.7283 - val_loss: 0.5957
Epoch 7/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 40s 25ms/step - accuracy: 0.8383 - loss: 0.3771 - val_accuracy: 0.9442 - val_loss: 0.1494
Epoch 8/10
895/895 ━━━━━━━━━━━━━━━━━━━━ 41s 25ms/step - accuracy: 0.9587 - loss: 0.1188 - 

In [89]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.6, 1, 0)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

448/448 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step
Accuracy: 0.9631674587643276


In [90]:
confusion_matrix(y_test, y_pred)

array([[6903,  178],
       [ 349, 6878]])

In [91]:
for i in range(10):
  print(f"True label: {y_test[i]}")
  print(f"Predicted label: {y_pred[i]}")

True label: 1
Predicted label: [1]
True label: 0
Predicted label: [0]
True label: 0
Predicted label: [0]
True label: 1
Predicted label: [1]
True label: 0
Predicted label: [0]
True label: 1
Predicted label: [1]
True label: 1
Predicted label: [1]
True label: 0
Predicted label: [0]
True label: 1
Predicted label: [1]
True label: 1
Predicted label: [1]


In [92]:

def predict_custom_text(text, model, vocab_size, sent_length):
  # Preprocess the text
  preprocessed_text = preprocess_txt(text)

  onehot_encoded = one_hot(preprocessed_text, vocab_size)
  padded_sequence = pad_sequences([onehot_encoded], padding='post', maxlen=sent_length)

  # Make a prediction
  prediction = model.predict(padded_sequence)

  # Return the predicted label (0 for fake, 1 for real)
  return int(prediction > 0.6)

# Example usage
custom_text = df['text'][2]
prediction = predict_custom_text(custom_text, model, vocab_size, sent_length)


if prediction == 0:
  print("The model predicts this news article is fake.")
else:
  print("The model predicts this news article is real.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
The model predicts this news article is fake.


<ipython-input-92-9da75d9cccc0>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(prediction > 0.6)
